In [1]:
# If we only form estimates from the prediction then the measurement will be ignored.
# If this is to work we need to take some kind of blend of the prediction and measurement

# the algorithm for gh in a pinch 

#     Multiple data points are more accurate than one data point, so throw nothing away no matter how inaccurate it is.
#     Always choose a number part way between two data points to create a more accurate estimate.
#     Predict the next measurement and rate of change based on the current estimate and how much we think it will change.
#     The new estimate is then chosen as part way between the prediction and next measurement scaled by how accurate each is.


In [2]:
# his is almost never true. Real sensors are more likely to get readings nearer the true value, and are less and less likely to get readings the further away from the true value it gets. We will cover this in detail in the Gaussian chapter.

In [4]:
# The system is the object that we want to estimate.

In [5]:
# The state of the system is the current configuration or values of that system that is of interest to us

In [6]:
# he measurement is a measured value of the system. Measurements can be inaccurate, so it may not have the same value as the state.

In [8]:
# The state estimate is our filter's estimate of the state.

In [9]:
# In other words, the state should be understood as the actual value of the system. This value is usually hidden to us. If I stepped on a scale you'd then have a measurement. We call this observable since you can directly observe this measurement. In contrast, you can never directly observe my weight, you can only measure it.

# This language of hidden and observable is important. Any estimation problem consists of forming an estimate of a hidden state via observable measurements. If you read the literature these terms are used when defining a problem, so you need to be comfortable with them.

In [10]:
# We use a process model to mathematically model the system. In this chapter our process model is the assumption that my weight today is yesterday's weight plus my weight gain for the last day. The process model does not model or otherwise account for the sensors.

In [11]:
# The system error or process error is the error in this model. We never know this value exactly; if we did we could refine our model to have zero error. Some texts use plant model and plant error. You may also see system model. They all mean the same thing.

In [13]:
# The predict step is known as system propagation. It uses the process model to form a new state estimate. Because of the process error this estimate is imperfect. Assuming we are tracking data over time, we say we propagate the state into the future. Some texts call this the evolution.

In [14]:
# The update step is known as the measurement update. One iteration of the system propagation and measurement update is known as an epoch.

In [15]:
# prediction vs measuremnt 

# In this chapter we will merely be assigning different values to $g$ and $h$ in a more intuitive, and thus less optimal way. But the fundamental idea is to blend somewhat inaccurate measurements with somewhat inaccurate models of how the systems behaves to get a filtered estimate that is better than either information source by itself.?

# We can express this as an algorithm:

# Initialization

# 1. Initialize the state of the filter
# 2. Initialize our belief in the state

# Predict

# 1. Use system behavior to predict state at the next time step
# 2. Adjust belief to account for the uncertainty in prediction

# Update

# 1. Get a measurement and associated belief about its accuracy
# 2. Compute residual between estimated state and measurement
# 3. New estimate is somewhere on the residual line

# We will use this same algorithm throughout the book, albeit with some modifications.

In [16]:
# generic gh filter algorithm

# weights = [158.0, 164.2, 160.3, 159.9, 162.1, 164.6, 
#            169.6, 167.4, 166.4, 171.0, 171.2, 172.6]  the data is a series of measuremnts that you will iterate over in a for loop


def g_h_filter(data, x0, dx, g, h, dt):
    """
    Performs g-h filter on 1 state variable with a fixed g and h.

    'data' contains the data to be filtered.
    'x0' is the initial value for our state variable
    'dx' is the initial change rate for our state variable
    'g' is the g-h's g scale factor 
    'h' is the g-h's h scale factor
    'dt' is the length of the time step 
    """
    
    x_est = x0
    results = []
    for z in data:     # z is measurements 
        
        #predict step
        x_pred = x_est + (dx*dt)   
        dx = dx
        
        # update step 
        residual = z - x_pred
        dx = dx + h * (residual) / dt     # h is the scaling for the change in measuemrn over time 
        x_est = x_pred + g * residual     # g is the caling we used for the measuremets
        
        # populate results
        results.append(x_est)

In [17]:
#  Each filter is differentiated by how $g$ and $h$ are chosen. So there is no 'one size fits all' advice that I can give here. Some filters set $g$ and $h$ as constants, others vary them dynamically. The Kalman filter varies them dynamically at each step. Some filters allow $g$ and $h$ to take any value within a range, others constrain one to be dependent on the other by some function $f(\dot{}), \mbox{where }g = f(h)$.

In [18]:
#  recognize that the typical formulation of the Kalman filter does not use $g$ and $h$ at all. The Kalman filter is a g-h filter because it mathematically reduces to this algorithm. When we design the Kalman filter we use design criteria that can be mathematically reduced to $g$ and $h$, but the Kalman filter form is usually a much more powerful way to think about the problem

In [19]:
# a bad inintal guess ca cause ringing  which menas that the singal overshoots and unershoot the data in a suidosdal type pattern

In [20]:
# UNDERSTANDING HOW VARYING G AND H AFFECT THE RESULTS
# This will give us strong insight into the fundamental strengths and limitations of this type of filter, and help us understand the behavior of the rather more sophisticated Kalman filter.

In [21]:
# GENERATE WHITE NOSE WITH NUMPY.RANDON.RAND()
from numpy.random import randn
def gen_data(x0, dx, count, noise_factor):
    return [x0 + dx*i + randn()*noise_factor for i in range(count)]

measurements = gen_data(0, 1, 30, 1)

In [24]:
# USE THE G-H FILTER ON THE GENERATED DATA FUNCITON ABOVE
data = g_h_filter(data=measurements, x0=0., dx=1., dt=1., g=.2, h=0.02)


In [25]:
### continue late
# the point
# 

# It is clear that as $g$ is larger we more closely follow the measurement instead of the prediction
# ow let's leave $g$ unchanged and investigate the effect of modifying $h$. We know that $h$ affects how much we favor the measurement of $\dot{x}$ vs our prediction
#  A smaller $h$ will cause us to react more slowly.

In [26]:
def compute_new_position(pos, vel, dt=1):
    return pos + (vel * dt)

def measure_position(pos):
        return pos + random.randn()*500
